<a href="https://colab.research.google.com/github/8Raouf24/PFE/blob/master/benchmarkRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install surprise

In [75]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import SVD,SVDpp,NMF

from surprise.model_selection import cross_validate

D'abord , nous récupérons notre dataset grace a la commande wget

In [13]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Musical_Instruments_5.json.gz

--2020-08-12 19:19:18--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Musical_Instruments_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39356138 (38M) [application/octet-stream]
Saving to: ‘Musical_Instruments_5.json.gz.1’

Musical_Instruments 100%[===================>]  37.53M  13.6MB/s    in 2.8s    

2020-08-12 19:19:21 (13.6 MB/s) - ‘Musical_Instruments_5.json.gz.1’ saved [39356138/39356138]



Nous pouvons manipuler les données sous forme de générateur , que nous utiliserons pour remplir un dictionnaire ( les données sont sous format .json)


In [25]:
### load the meta data

data = []
with gzip.open('Musical_Instruments_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

231392
{'overall': 5.0, 'verified': True, 'reviewTime': '10 30, 2016', 'reviewerID': 'A3FO5AKVTFRCRJ', 'asin': '0739079891', 'reviewerName': 'francisco', 'reviewText': "It's good for beginners", 'summary': 'Five Stars', 'unixReviewTime': 1477785600}


Notre pré-traitement consiste a récolter seulement les colonnes de notre DataFrame qui nous serviront a la recommandation , c-à-d les items , les users , et les ratings

In [26]:
# we create a new data frame containing the required informations for our benchmark

df = pd.DataFrame.from_dict(data)
min_rating = 1.0
max_rating = 5.0
data_dict = {'userID':df['reviewerID'],'itemID':df['asin'],'rating':df['overall']}
df_rs = pd.DataFrame.from_dict(data_dict)



print(df_rs.info)

<bound method DataFrame.info of                 userID      itemID  rating
0       A3FO5AKVTFRCRJ  0739079891     5.0
1       A3UCGC1DHFMBCE  0739079891     5.0
2       A2S9SLRYLPGYZB  0739079891     5.0
3       A15RTJWPG8OKOE  0739079891     4.0
4       A12ET1WO3OAVU7  0739079891     1.0
...                ...         ...     ...
231387  A2GLR2Q3M1PC7P  B01HIDOPP2     5.0
231388  A3S1LSTRW57BP9  B01HIDOPP2     5.0
231389  A2BNWRRADDBSVO  B01HIDOPP2     4.0
231390  A1KRL5ZRON6DPT  B01HIDOPP2     1.0
231391  A3QN3W0PJ1DXIT  B01HIDOPP2     5.0

[231392 rows x 3 columns]>


Nous préparons notre reader et nous chargeons notre data frame sous un format "Dataset" pour l'utilisation de surprise

In [20]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df_rs, reader)

Préparation du training set

In [66]:
train=df_rs.sample(frac=0.8,random_state=200) #random state is a seed value
data = Dataset.load_from_df(train, reader)
trainset = data.build_full_trainset()

In [56]:
import numpy as np
from sklearn.metrics import mean_squared_error 



1.   Algorithme SVD




In [42]:
S = SVD()
S.fit(trainset)

**Train**

In [74]:
list_result=[]
for i,u in zip(train['userID'],train['itemID']):
  result = S.predict(u,i)
  list_result.append(result[3])
list_result = np.array(list_result)
tr_result = np.array(train['rating'])
mean_squared_error(tr_result,list_result)



0.9642320880165741

Test

In [73]:
import math
list_result=[]
for i,u in zip(test['userID'],test['itemID']):
  result = S.predict(u,i)
  list_result.append(result[3])
list_result = np.array(list_result)
tr_result = np.array(test['rating'])
mean_squared_error(tr_result,list_result)



0.9606097742874777



2.   Algorithme SVD ++




In [82]:
Spp = SVDpp()
Spp.fit(trainset)

In [87]:
list_result=[]
for i,u in zip(train['userID'],train['itemID']):
  result = Spp.predict(u,i)
  list_result.append(result[3])
list_result = np.array(list_result)
tr_result = np.array(train['rating'])
mean_squared_error(tr_result,list_result)


0.9642320880165741

In [86]:
import math
list_result=[]
for i,u in zip(test['userID'],test['itemID']):
  result = Spp.predict(u,i)
  list_result.append(result[3])
list_result = np.array(list_result)

tr_result = np.array(test['rating'])
mean_squared_error(tr_result,list_result)


[4.47081798 4.47081798 4.47081798 4.47081798 4.47081798]


0.9606097742874777



3.   Algorithme NMF




In [79]:
N = NMF()
N.fit(trainset)

In [80]:
list_result=[]
for i,u in zip(train['userID'],train['itemID']):
  result = N.predict(u,i)
  list_result.append(result[3])
list_result = np.array(list_result)
tr_result = np.array(train['rating'])
mean_squared_error(tr_result,list_result)

0.9642320880165741

In [81]:
import math
list_result=[]
for i,u in zip(test['userID'],test['itemID']):
  result = N.predict(u,i)
  list_result.append(result[3])
list_result = np.array(list_result)
tr_result = np.array(test['rating'])
mean_squared_error(tr_result,list_result)


0.9606097742874777